In [88]:
import pandas as pd
import pandas as pd
import numpy as np
import re
import torch
import os
import torch.nn.functional as F
import random  
from spacy.lang.en import English
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

device = "cpu"

nlp = English()

nlp.add_pipe("sentencizer")

doc = nlp("This is a sentence. This another sentence. I like Elephants")

def assign_properties(pages_and_text, column = "Page Content"):
    result = []
    print(len(pages_and_text[column].tolist()))
    for index, page in enumerate(pages_and_text[column].tolist()):
        result.append({"page_number": index,
                               "page_char_count": len(page),
                               "page_word_count": len(page.split(" ")),
                               "page_sentences": len(page.split(".")),
                               "page_token_count": len(page)/4, # 1 token ~ 4 characters
                                "text": page
                               })
    return result
def splitting_into_sentences(pages_and_texts):
    for item in tqdm(pages_and_texts):
        item["sentences"] = list(nlp(item["text"].sents))
        item["sentences"] = [str(sentence) for sentence in item["sentences"]]
        item["page_space_count_spacy"] = len(item["sentences"])
    return pages_and_texts


def split_list(input_list: list, slice_size: int) -> list[list[str]]:
    return [input_list[i : i + slice_size] for i in range(0, len(input_list), slice_size)]


def chunk(pages_and_texts, num_sentence_chuck_size = 10):
    for item in tqdm(pages_and_texts):
        item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chuck_size)
        item["num_chunk"] = len(item["sentence_chunks"])


def join_sentences(pages_and_texts):
    pages_and_chunks = []
    for item in tqdm(pages_and_texts):
        for sentence_chunk in item["sentence_chunks"]:
            chunk_dict = {}
            chunk_dict["page_number"] = item["page number"]

            # Join the sentences, so that the sentence start with capital letter after period
            joined_sentence_chunk = "".join(sentence_chunk).replace(" ", " ").strip()
            joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)

            chunk_dict["sentence_chunk"] = joined_sentence_chunk

            chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
            chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
            chunk_dict["chunk_token_count"] = len(joined_sentence_chunk)/4

            pages_and_chunks.append(chunk_dict)
    return pages_and_chunks

def get_embeddings(text):
    embedding_model = SentenceTransformer(model_name_or_path= "all-mpnet-base-v2",
                                      device=device)
    embeddings = embedding_model.encode(text)
    return embeddings


In [89]:
pages_and_text = []
df_smus_website = pd.read_csv("smus_page.csv") 
df_smus_handbook = pd.read_csv("smus_handbook.csv")
for page in df_smus_website["Page Content"]:
    pages_and_text.append(page.replace("\n", " "))
for page in df_smus_handbook["Page Content"]:
    pages_and_text.append(page)

df = pd.DataFrame(pages_and_text, columns = ["Page Content"])
df.head()

,Page Content
0,Explore Cookie Settings When you visit any...
1,Breadcrumb Start Here Thank you for choosin...
2,Breadcrumb Admissions Publications If you ...
3,Breadcrumb Middle School The Middle School ...
4,Schaffter Hall for music (left) is home to ...


In [90]:
pages_and_text = assign_properties(df)

145


In [91]:
df = pd.DataFrame(pages_and_text)
df

,page_number,page_char_count,page_word_count,page_sentences,page_token_count,text
0,0,1496,241,14,374.00,Explore Cookie Settings When you visit any...
1,1,3488,567,30,872.00,Breadcrumb Start Here Thank you for choosin...
2,2,1169,194,10,292.25,Breadcrumb Admissions Publications If you ...
3,3,2809,462,20,702.25,Breadcrumb Middle School The Middle School ...
4,4,2167,345,15,541.75,Schaffter Hall for music (left) is home to ...
...,...,...,...,...,...,...
140,140,5224,863,44,1306.00,St. Michaels University SchoolFamily Handbook ...
141,141,5093,838,52,1273.25,St. Michaels University SchoolFamily Handbook ...
142,142,1056,188,13,264.00,St. Michaels University SchoolFamily Handbook ...
143,143,5416,861,44,1354.00,St. Michaels University SchoolFamily Handbook ...


In [92]:
pages_and_text = splitting_into_sentences(pages_and_text)

  0%|          | 0/145 [00:00<?, ?it/s]


AttributeError: 'str' object has no attribute 'sents'